In [7]:
# Baixando e Instalando os modelos e pacotes necessários
%pip install transformers einops accelerate bitsandbytes

# Transformers: Processamento de linguagem natural, classificação, tradução e geraão de texto

# Einops: Manipulação de tensores (arrays multidimensionais)

# Accelerate: Treinamento distribuído e paralelo, reduz a complexidade do treinamento

# Bitsandbytes: Manipulação de bits e bytes, manupulação de arquivos binários e serialização

Note: you may need to restart the kernel to use updated packages.


In [8]:
# AutoModelForCausalLM: Modelo de linguagem causal, geração de texto
# AutoTokenizer: Tokenizador de texto, transforma texto em tokens
# pipeline: Processamento de texto, tokenização, geração de texto, tradução, classificação, etc
# BitsAndBytesConfig: Configuração para manipulação de bits e bytes

from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, BitsAndBytesConfig

In [9]:
# torch: Biblioteca de aprendizado de máquina, manipulação de tensores, cálculos matemáticos

import torch

device = "cuda:0" if torch.cuda.is_available() else "cpu"

In [10]:
device

'cuda:0'

In [11]:
# Configuração do modelo e tokenizador estatico para geração de texto
torch.random.manual_seed(42)

In [12]:
%pip install python-dotenv

import os
from dotenv import load_dotenv

load_dotenv()

# Guardar token
os.environ["HF_TOKEN"] = os.getenv("HF_TOKEN")

Note: you may need to restart the kernel to use updated packages.


In [13]:
# libera memoria da GPU
torch.cuda.empty_cache()
torch.cuda.reset_peak_memory_stats()
torch.cuda.reset_accumulated_memory_stats()
# torch.cuda.reset_max_memory_allocated() - obsoleto
# torch.cuda.reset_max_memory_cached() - obsoleto

In [14]:
id_modelo = "microsoft/Phi-3-mini-4k-instruct"

In [15]:
%pip install tiktoken

# Definindo a variável de ambiente para evitar fragmentação 
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

# Desabilita o paralelismo na biblioteca tokenizers para evitar problemas de deadlock
os.environ["TOKENIZERS_PARALLELISM"] = "false"

from transformers import BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(load_in_8bit=True)  # Ativa a quantização para 8 bits


# criando o modelo
model = AutoModelForCausalLM.from_pretrained(
    id_modelo,
    # device_map = 'balanced', # Usando a GPU/CPU automaticamente 
    device_map = 'cuda', # Usando a GPU
    # torch_dtype = 'auto', # Ajusta automaticamente os tipos de dados dos tensores do modelo
    torch_dtype = torch.float16, # Tipo de dado do modelo
    trust_remote_code = True, # Permite o carregamento personalizado de modelos
    # attn_implementation="eager", # metodo de implementação pro metodo de atenção
    quantization_config=quantization_config,
    )

Note: you may need to restart the kernel to use updated packages.


`flash-attention` package not found, consider installing for better performance: No module named 'flash_attn'.
Current `flash-attention` does not support `window_size`. Either upgrade or use `attn_implementation='eager'`.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [16]:
# criando o tokenizador
tokenizer = AutoTokenizer.from_pretrained(
    id_modelo, 
    trust_remote_code = True # Permite o carregamento personalizado de modelos
    )

In [17]:
# criação do pipeline
pipe = pipeline(
    "text-generation", 
    model = model, 
    tokenizer = tokenizer
    )

Device set to use cuda


In [18]:
# parametros de geração de texto
generation_args = {
    "max_new_tokens": 500, # comprimento do texto de entrada
    "return_full_text": False, # determina se retorna todo o texto ou apenas o que foi gerado
    "temperature": 0.1, # controla a aleatoriedade da geração de texto
    "do_sample": True, # determina se a aleatoriedade é usada na geração de texto
    "num_beams": 3, # Número de feixes para busca 
    "early_stopping": True, # Pare a geração assim que uma sentença completa é gerada
    "truncation": True # Ativar a truncação explicitamente
}

In [19]:
prompt = "Explique o que é computação quântica"
output = pipe(prompt, **generation_args)

The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.
`get_max_cache()` is deprecated for all Cache classes. Use `get_max_cache_shape()` instead. Calling `get_max_cache()` will raise error from v4.48
You are not running the flash-attention implementation, expect numerical differences.


In [20]:
output

[{'generated_text': ' e como ela difere da computação clássica.\n\nComputação quântica é um campo da ciência da computação que utiliza os princípios da mecânica quântica para processar informações. Ao contrário da computação clássica, que usa bits binários (0 ou 1) para representar dados, a computação quântica usa qubits, que podem existir em um estado de 0, 1 ou ambos (superposição). Isso permite que os computadores quânticos realizem muitas operações simultaneamente, potencialmente acelerando significativamente certos tipos de cálculos.\n\nUma das principais diferenças entre computação quântica e clássica é o conceito de entrelaçamento quântico. Quando dois qubits estão entrelaçados, o estado de um qubit depende do estado do outro, independentemente da distância entre eles. Isso permite que os computadores quânticos realizem cálculos complexos de forma mais eficiente do que os computadores clássicos.\n\nOutra diferença chave é o uso de portas lógicas quânticas, que manipulam os estad

In [21]:
print(output[0]['generated_text'])

 e como ela difere da computação clássica.

Computação quântica é um campo da ciência da computação que utiliza os princípios da mecânica quântica para processar informações. Ao contrário da computação clássica, que usa bits binários (0 ou 1) para representar dados, a computação quântica usa qubits, que podem existir em um estado de 0, 1 ou ambos (superposição). Isso permite que os computadores quânticos realizem muitas operações simultaneamente, potencialmente acelerando significativamente certos tipos de cálculos.

Uma das principais diferenças entre computação quântica e clássica é o conceito de entrelaçamento quântico. Quando dois qubits estão entrelaçados, o estado de um qubit depende do estado do outro, independentemente da distância entre eles. Isso permite que os computadores quânticos realizem cálculos complexos de forma mais eficiente do que os computadores clássicos.

Outra diferença chave é o uso de portas lógicas quânticas, que manipulam os estados quânticos dos qubits. Es

In [22]:
prompt = "Quanto é 7 + 6 / 2"
output = pipe(prompt, **generation_args)
print(output[0]['generated_text'])

 - 3?

# Answer:Para resolver a expressão 7 + 6 / 2 - 3, precisamos seguir a ordem das operações, que é Parênteses, Expoentes, Multiplicação e Divisão (da esquerda para a direita), Adição e Subtração (da esquerda para a direita), também conhecida como PEMDAS.

1. Primeiro, realizamos a divisão: 6 / 2 = 3
2. Em seguida, realizamos a adição: 7 + 3 = 10
3. Por fim, realizamos a subtração: 10 - 3 = 7

Portanto, a resposta é 7.


In [26]:
prompt = "Quem foi a primeira pessoa a ir pro espaço?"

In [27]:
# template
template = """<|system|>
You are a helpful assistant.<|end>

<|user|>
"{}"<|end|>
<|assistant|>""".format(prompt)

In [24]:
template

'<|system|>\nYou are a helpful assistant.<|end>\n\n<|user|>\n"Quanto é 7 + 6 / 2"<|end|>\n<|assistant|>'

In [28]:
output = pipe(template, **generation_args)
print(output[0]['generated_text'])

 A primeira pessoa a ir ao espaço foi Yuri Gagarin, um cosmonauta soviético. Ele completou uma órbita ao redor da Terra em 12 de abril de 1961, a bordo da nave espacial Vostok 1, tornando-se um ícone mundial da exploração espacial.


In [29]:
prompt = "Qual é a capital do Brasil?" # @param {type: string}

template = """<|system|>
You are a helpful assistant.<|end|>
<|user|>
"{}"<|end|>
<|assistant|>""".format(prompt)

output = pipe(template, **generation_args)
print(output[0]['generated_text'])

 A capital do Brasil é Brasília. Foi inaugurada em 21 de abril de 1960, substituindo a antiga capital, Rio de Janeiro. A cidade foi projetada pelo arquiteto Oscar Niemeyer e o urbanista Lúcio Costa, com o objetivo de promover o desenvolvimento econômico e político do interior do país.
